# Preprocess c file

TCI percentiles per city-country

`Last update`: 2021-04-15

In [532]:
import subprocess
import pandas as pd
from loguru import logger 

def _create_cs_file(path_c):
    
    df = pd.read_csv(path_c)
    a = set(df.region_slug)
    
    df = df \
        .drop_duplicates('region_slug') 
    b = set(df.region_slug)
    
    df = df\
        .dropna()
    c = set(df.region_slug)

    print (a.difference(b))
    print (a.difference(c))
    
    for k in df.columns:
        if k != 'region_slug':
            df[k] = pd.to_numeric(df[k])
        if k == 'region_slug':
            df[k] = df[k].astype(str) 

    df_c = df \
        .assign(c_low_p01 = lambda x: (x['q25']-x['q01'])/(x['q75']-x['q25']),
                c_upp_p99 = lambda x: (x['q99']-x['q75'])/(x['q75']-x['q25']),
                c_low_p05 = lambda x: (x['q25']-x['q05'])/(x['q75']-x['q25']),
                c_upp_p95 = lambda x: (x['q95']-x['q75'])/(x['q75']-x['q25'])
               ) \
        [['region_slug', 'c_low_p01', 'c_upp_p99', 'c_low_p05', 'c_upp_p95']]
    
    df_c['c_min'] = df_c[['c_low_p01', 'c_upp_p99', 'c_low_p05', 'c_upp_p95']].min(axis=1)
    df_c['c_max'] = df_c[['c_low_p01', 'c_upp_p99', 'c_low_p05', 'c_upp_p95']].max(axis=1)
    df_c['c_avg'] = df_c[['c_low_p01', 'c_upp_p99', 'c_low_p05', 'c_upp_p95']].mean(axis=1)
    
    return df_c

def _save_cs_file(): 
    
    logger.debug('writing file')
    
    df_c = _create_cs_file(path_c = '~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv')
    print(df_c.shape)
    print(df_c.describe())
    df_c.to_csv('~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_c_iqr.csv', index=False)
    

## Copy percentile

In [533]:
!cp ~/shared/spd-sdv-omitnik-waze/waze_tci/region_quantiles/quantiles/regions_quantiles-20210420.csv ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/raw/regions_quantiles.csv

!sed "s/\[//g;s/\]//g;s/['\"]//g" ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/raw/regions_quantiles.csv > ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr_00.csv

In [534]:
%%bash 

echo -e "region_slug,mean,var_p, var_s,q10,q20,q30,q40,q50,q60,q70,q80,q90,q01,q99,q25,q75,q05,q95" | cat - ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr_00.csv > ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv

In [535]:
!rm ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr_00.csv

In [536]:
!head -2 ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv

region_slug,mean,var_p, var_s,q10,q20,q30,q40,q50,q60,q70,q80,q90,q01,q99,q25,q75,q05,q95
aguascalientes,8039136.868743886,8862843413565.455,8887326406420.607,3829261.1993683483,5605996.229991711,6600013.058713919,6932325.881132038,7473441.530126513,8626943.901942156,9718372.086622398,11136396.36923769,11136396.36923769,2151065.466800198,14393234.680959623,6215087.576892198,10248917.983242217,3349729.7486610203,12964231.367048915


In [537]:
!wc -l ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv

107 /home/soniame/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv


## Save c's file

In [519]:
_save_cs_file()

2021-04-22 13:23:07.285 | DEBUG    | __main__:_save_cs_file:43 - writing file


set()
{nan}
(100, 8)
        c_low_p01   c_upp_p99   c_low_p05   c_upp_p95       c_min       c_max  \
count  100.000000  100.000000  100.000000  100.000000  100.000000  100.000000   
mean     0.872206    1.501374    0.654722    0.771653    0.531642    1.572998   
std      0.423429    0.887724    0.342507    0.374849    0.220602    0.864893   
min      0.213163    0.506459    0.165875    0.308937    0.165875    0.506459   
25%      0.525674    0.965353    0.400885    0.517304    0.374315    1.034190   
50%      0.737493    1.279874    0.556730    0.674209    0.494360    1.407540   
75%      1.234659    1.694937    0.875145    0.900741    0.658642    1.788816   
max      2.017849    5.851183    1.580424    2.572249    1.482831    5.851183   

            c_avg  
count  100.000000  
mean     0.949989  
std      0.347770  
min      0.388697  
25%      0.683010  
50%      0.910712  
75%      1.147068  
max      2.285736  


In [538]:
df = pd.read_csv('~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_c_iqr.csv')

In [539]:
df[df.region_slug == 'salvador']
#df[75:]

,region_slug,c_low_p01,c_upp_p99,c_low_p05,c_upp_p95,c_min,c_max,c_avg
58,salvador,0.726099,0.757092,0.567365,0.360343,0.360343,0.757092,0.602725


In [540]:
print(len(df.region_slug.to_list()))
#df.region_slug.to_list()

100


### Regions missing

In [541]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2
#from pathlib import Path
#home = str(Path.home())

import sys
sys.path.insert(0, '../')

from src import utils

In [542]:
conn = utils.connect_athena(path='../configs/athena.yaml')

../src/utils.py:158: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  athena_config = yaml.load(open(path, "r"))


In [543]:
qry = """
    select distinct region_slug
    from spd_sdv_waze_corona.prod_daily_daily_index
"""
dash_regions = pd.read_sql_query(qry, conn).sort_values('region_slug').region_slug.unique()

In [544]:
cpar_regions = df.region_slug.to_list()

In [545]:
a = set(dash_regions)
b = set(cpar_regions)
len(a.intersection(b))

72

In [546]:
a.difference(b)

{'br_states_acre',
 'br_states_alagoas',
 'br_states_amapa',
 'br_states_amazonas',
 'br_states_bahia',
 'br_states_ceara',
 'br_states_distrito_federal',
 'br_states_espirito_santo',
 'br_states_goias',
 'br_states_maranhao',
 'br_states_mato_grosso',
 'br_states_mato_grosso_do_sul',
 'br_states_minas_gerais',
 'br_states_para',
 'br_states_paraiba',
 'br_states_parana',
 'br_states_pernambuco',
 'br_states_piaui',
 'br_states_rio_de_janeiro',
 'br_states_rio_grande_do_norte',
 'br_states_rio_grande_do_sul',
 'br_states_rondonia',
 'br_states_roraima',
 'br_states_santa_catarina',
 'br_states_sao_paulo',
 'br_states_sergipe',
 'br_states_tocantins',
 'country_brazil',
 'country_honduras',
 'country_mexico',
 'country_nicaragua',
 'country_panama',
 'country_paraguay',
 'country_peru',
 'country_trinidad_and_tobago',
 'country_uruguay'}